In [ ]:
import sys
import pandas as pd
import numpy as np

def error(msg):
    print("Error:", msg)
    sys.exit(1)

def topsis(input_file, weights, impacts, output_file):

    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        error("Input file not found")

    if df.shape[1] < 3:
        error("Input file must contain at least three columns")

    data = df.iloc[:, 1:].copy()

    try:
        data = data.astype(float)
    except ValueError:
        error("Columns from 2nd to last must contain numeric values only")

    weights = weights.split(',')
    impacts = impacts.split(',')

    if len(weights) != data.shape[1]:
        error("Number of weights must match number of criteria")

    if len(impacts) != data.shape[1]:
        error("Number of impacts must match number of criteria")

    try:
        weights = np.array(weights, dtype=float)
    except ValueError:
        error("Weights must be numeric")

    for i in impacts:
        if i not in ['+', '-']:
            error("Impacts must be '+' or '-'")

    norm = np.sqrt((data ** 2).sum())
    normalized = data / norm
    weighted = normalized * weights

    ideal_best, ideal_worst = [], []

    for i in range(len(impacts)):
        if impacts[i] == '+':
            ideal_best.append(weighted.iloc[:, i].max())
            ideal_worst.append(weighted.iloc[:, i].min())
        else:
            ideal_best.append(weighted.iloc[:, i].min())
            ideal_worst.append(weighted.iloc[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    d_best = np.sqrt(((weighted - ideal_best) ** 2).sum(axis=1))
    d_worst = np.sqrt(((weighted - ideal_worst) ** 2).sum(axis=1))

    score = d_worst / (d_best + d_worst)

    df['Topsis Score'] = score
    df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)

    df.to_csv(output_file, index=False)
    return df


In [ ]:
sample_data = {
    'Fund Name': ['M1','M2','M3','M4','M5'],
    'P1': [0.67,0.6,0.82,0.6,0.76],
    'P2': [0.45,0.36,0.67,0.36,0.58],
    'P3': [6.5,3.6,3.8,6.9,4.8],
    'P4': [42.6,53.3,63.1,69.2,43],
    'P5': [12.56,14.47,17.1,18.42,12.29]
}

df = pd.DataFrame(sample_data)
df.to_csv("data.csv", index=False)
df


,Fund Name,P1,P2,P3,P4,P5
0,M1,0.67,0.45,6.5,42.6,12.56
1,M2,0.60,0.36,3.6,53.3,14.47
2,M3,0.82,0.67,3.8,63.1,17.10
3,M4,0.60,0.36,6.9,69.2,18.42
4,M5,0.76,0.58,4.8,43.0,12.29


In [ ]:
result = topsis("data.csv", "1,1,1,2,1", "+,+,-,+,+", "output-result.csv")
result


,Fund Name,P1,P2,P3,P4,P5,Topsis Score,Rank
0,M1,0.67,0.45,6.5,42.6,12.56,0.147821,5
1,M2,0.60,0.36,3.6,53.3,14.47,0.442856,3
2,M3,0.82,0.67,3.8,63.1,17.10,0.835021,1
3,M4,0.60,0.36,6.9,69.2,18.42,0.527635,2
4,M5,0.76,0.58,4.8,43.0,12.29,0.371895,4
